In [1]:
!pip3 install -q -U datasets
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U jinja2
!pip3 install -q -U protobuf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.43.2 requires protobuf<6,>=3.20, but you have protobuf 6.30.1 which is incompatible.


In [2]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

BASE_MODEL = "google/gemma-1.1-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from datasets import load_dataset
dataset = load_dataset("kyujinpy/KOpen-platypus")

README.md:   0%|          | 0.00/6.83k [00:00<?, ?B/s]

(…)-00000-of-00001-4de27a57725b7443.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24926 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'data_source'],
        num_rows: 24926
    })
})

In [7]:
dataset['train'][0]

{'input': '',
 'output': '모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \\frac{1}{3}$, $P(B) = \\frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \\frac{1}{3} - frac{5}{12} = \\frac{12}{12} - frac{4}{12} - frac{5}{12} = \\frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \\frac{1}{4}$입니다.',
 'instruction': '보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.',
 'data_source': 'MATH/PRM-800K'}

In [8]:
train_data = dataset['train']

In [9]:
messages = [
    {"role": "user",
        "content": train_data[0]['instruction']
    }
]

chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
chat_message


'<bos><start_of_turn>user\n보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.<end_of_turn>\n<start_of_turn>model\n'

In [10]:
from transformers import TextStreamer, GenerationConfig

streamer = TextStreamer(tokenizer)

def gen(prompt):
    generation_config = GenerationConfig(
        temperature = 1.0,
        max_new_tokens = 512,
        exponential_decay_length_penalty = (512, 1.03),
        eos_token_id = tokenizer.eos_token_id,
        repetition_penalty = 1.17,
        do_sample = True,
        top_k = 49,
        top_p = 0.14,
        min_length = 5,
        use_cache = True,
        return_dict_in_generate = True
    )
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ).to("cuda"),
        generation_config=generation_config,
        streamer=streamer
    )

In [11]:
gen(chat_message)

`generation_config` default values have been modified to match model-specific defaults: {'pad_token_id': 0, 'bos_token_id': 2, 'transformers_version': '4.38.1'}. If this is not desired, please set these values explicitly.


<bos><bos><start_of_turn>user
보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.<end_of_turn>
<start_of_turn>model
스피너가 $C$에 떨어질 확률은 다음과 같습니다.

$$P(C) = P(A) + P(B) - P(A and B)$$
$$= \frac{1}{3} + \frac{5}{12}$$
$$= \frac{7}{12}$$<eos>


In [12]:
def format_prompt(data):
    messages = [
        {
            "role": "user",
            "content": data['instruction']
        },
        {
            "role": "model",
            "content": data['output']
        }
    ]
    chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    chat_message += '<eos>'
    return chat_message

In [13]:
chat_message = format_prompt(train_data[0])
chat_message

'<bos><start_of_turn>user\n보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.<end_of_turn>\n<start_of_turn>model\n모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \\frac{1}{3}$, $P(B) = \\frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \\frac{1}{3} - frac{5}{12} = \\frac{12}{12} - frac{4}{12} - frac{5}{12} = \\frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \\frac{1}{4}$입니다.<end_of_turn>\n<eos>'

In [14]:
training_args = TrainingArguments(
    output_dir="outputs",
#    num_train_epochs = 1,
    max_steps=3000,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    #warmup_steps=0.03,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    push_to_hub=False,
    report_to='none',
)

In [15]:
train_data

Dataset({
    features: ['input', 'output', 'instruction', 'data_source'],
    num_rows: 24926
})

In [16]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
    formatting_func=format_prompt
)

/tmp/ipykernel_61885/582958899.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Applying formatting function to train dataset:   0%|          | 0/24926 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/24926 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/24926 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/24926 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/24926 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [17]:
trainer.train()

Step,Training Loss
100,1.710100
200,1.425800
300,1.323400
400,1.337300


KeyboardInterrupt: 